In [529]:
from bs4 import BeautifulSoup
from faker import Faker
import requests
import random
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()
import pandas as pd
from datetime import datetime


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/isabelsearleriesgo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [381]:
def add_fake_messages(num_of_msg):
    try:
        messages = [faker.text(max_nb_chars=40) for i in range(num_of_msg)]
    except TypeError:
        raise TypeError("Enter an integer")
        users = requests.get("http://127.0.0.1:5000/table/users")
        for i in messages:
            iduser=random.randint(0,len(users.json()))
            url = f"http://127.0.0.1:5000/messages?idusers={iduser}&message_content={i}"
        return requests.get(url)

In [409]:
add_fake_messages(3)

In [406]:
def add_messages(message):
    users = requests.get("http://127.0.0.1:5000/table/users")
    iduser=random.randint(0,len(users.json()))
    url = f"http://127.0.0.1:5000/messages?idusers={iduser}&message_content={message}"
    return requests.get(url)

In [408]:
add_messages('I felt in love with the boss')

<Response [200]>

In [403]:
def add_messages_by_dept(message, dept):
    users = requests.get("http://127.0.0.1:5000/table/users")
    df_users=pd.DataFrame(users.json())
    us=df_users[df_users['department']==dept]['idusers'].tolist()
    iduser=random.choice(us)
    url = f"http://127.0.0.1:5000/messages?idusers={iduser}&message_content={message}"
    return requests.get(url)

In [404]:
add_messages_by_dept('I hate this job', 'Marketing')

<Response [200]>

In [384]:
def get_all_messages():
    url = "http://127.0.0.1:5000/table/messages"
    text = requests.get(url)
    to_analize = [i['message_content'] for i in text.json()] 
    return to_analize

In [414]:
get_all_messages()[-3:]

['Catch want two form TV develop above.',
 'I hate this job',
 'I felt in love with the boss']

In [415]:
def get_messages_by_dept(dept):
    try:
        pass
    except NameError:
        dept=str(dept)
        pass
    messages = requests.get("http://127.0.0.1:5000/table/messages")
    users = requests.get("http://127.0.0.1:5000/table/users")
    df_messages=pd.DataFrame(messages.json())
    df_users=pd.DataFrame(users.json())
    df_merged=df_users.merge(df_messages)
    df_merged['date_time'] = [i['$date']/1000 for i in df_merged['date_time']]
    df_merged['date_time'] = df_merged['date_time'].map(lambda x: datetime.fromtimestamp(x).strftime("%d/%m/%Y, %H:%M:%S")) 
    return list(df_merged[df_merged['department']==dept]['message_content'])
    

In [416]:
get_messages_by_dept('Marketing')

['I hate this job']

In [417]:
def get_messages_by_date(date):
    if type(date) == str:
        if "-" in date:
            date=date.replace("-","/")
        else:
            pass
        messages = requests.get("http://127.0.0.1:5000/table/messages")
        users = requests.get("http://127.0.0.1:5000/table/users")
        df_messages=pd.DataFrame(messages.json())
        df_users=pd.DataFrame(users.json())
        df_merged=df_users.merge(df_messages)
        df_merged['date_time'] = [i['$date']/1000 for i in df_merged['date_time']]
        df_merged['date_time'] = df_merged['date_time'].map(lambda x: datetime.fromtimestamp(x).strftime("%d/%m/%Y")) 
        return list(df_merged[df_merged['date_time']==date]['message_content'])
    else:
        raise TypeError("Enter a string %d/%m/%Y")      

In [418]:
get_messages_by_date('30/11/2020')

['I felt in love with the boss',
 'Catch want two form TV develop above.',
 'I hate this job']

In [419]:
def global_happiness():
    url = "http://127.0.0.1:5000/table/messages"
    text = requests.get(url)
    to_analize = [i['message_content'] for i in text.json()]
    sentiment=[sia.polarity_scores(i) for i in to_analize]
    pos=0
    neg=0
    for i in sentiment:
        pos+=i['pos']
        neg+=i['neg']
    mean_pos=pos/(len(sentiment))
    mean_neg=neg/(len(sentiment))
    if mean_pos < mean_neg:
        return print("Your employees are unhappy 😔")
    else:
        return print("Your employees are happy! 😃")

In [420]:
global_happiness()

Your employees are happy! 😃


In [421]:
def happiness_by_dept(dept):
    messages = requests.get("http://127.0.0.1:5000/table/messages")
    users = requests.get("http://127.0.0.1:5000/table/users")
    df_messages=pd.DataFrame(messages.json())
    df_users=pd.DataFrame(users.json())
    df_merged=df_users.merge(df_messages)
    df_merged['date_time'] = [i['$date']/1000 for i in df_merged['date_time']]
    df_merged['date_time'] = df_merged['date_time'].map(lambda x: datetime.fromtimestamp(x).strftime("%d/%m/%Y, %H:%M:%S")) 
    to_analize=list(df_merged[df_merged['department']==dept]['message_content'])
    sentiment=[sia.polarity_scores(i) for i in to_analize]
    pos=0
    neg=0
    for i in sentiment:
        pos+=i['pos']
        neg+=i['neg']
    mean_pos=pos/(len(sentiment))
    mean_neg=neg/(len(sentiment))
    if mean_pos < mean_neg:
        return print("Your team is unhappy 😔")
    else:
        return print("Your team is happy! 😃")

In [422]:
happiness_by_dept('Marketing')

Your team is unhappy 😔


In [423]:
def get_complet_table():
    messages = requests.get("http://127.0.0.1:5000/table/messages")
    users = requests.get("http://127.0.0.1:5000/table/users")
    df_messages=pd.DataFrame(messages.json())
    df_users=pd.DataFrame(users.json())
    df_merged=df_users.merge(df_messages)
    df_merged['date_time'] = [i['$date']/1000 for i in df_merged['date_time']]
    df_merged['date_time'] = df_merged['date_time'].map(lambda x: datetime.fromtimestamp(x).strftime("%d/%m/%Y, %H:%M:%S"))
    return df_merged

In [425]:
get_complet_table().head(3)

,idusers,name,lastname,department,idmessages,message_content,date_time
0,1,Pepe,Ruiz,IT,1,holaquetal,"28/11/2020, 20:29:56"
1,1,Pepe,Ruiz,IT,2,holaquetal,"28/11/2020, 20:29:58"
2,1,Pepe,Ruiz,IT,3,holaquetal,"28/11/2020, 20:32:13"


# Elthon John "Tiny Dancer"

In [620]:
def messages_love():
    url = "https://genius.com/Elton-john-tiny-dancer-lyrics"
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    song = soup.select('.Lyrics__Container-sc-1ynbvzw-2.jgQsqn')
    lyric=song[0].text
    return [lyric[i:i+40] for i in range(0, len(lyric), 40)]

# Elvis song "Only You"

In [615]:
def messages_mix():
    url = "https://www.allthelyrics.com/lyrics/elvis_presley/only_you-lyrics-423192.html"
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    song = soup.select('.content-text-inner')
    lyric="".join(song[0].text.split('\n'))
    return [lyric[i:i+40] for i in range(0, len(lyric), 40)]

# The All American Rejects "Gives you hell"

In [569]:
def messages_hate():
    url = "https://genius.com/The-all-american-rejects-gives-you-hell-lyrics"
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    song = soup.select('.Lyrics__Container-sc-1ynbvzw-2.jgQsqn')
    lyric=song[2].text
    return [lyric[i:i+40] for i in range(0, len(lyric), 40)]

# Marketing

In [603]:
marketing = messages_hate()
for j in marketing:
    add_messages_by_dept(j, 'Marketing')

# BI

In [622]:
bi = messages_love()
for j in bi:
    add_messages_by_dept(j, 'BI')

# Operations

In [624]:
operations = messages_mix()
for j in operations:
    add_messages_by_dept(j, 'Operations')